In [1]:
from __future__ import division
import os
import time
import math
import numpy as np
import tensorflow as tf
import tensorflow.contrib.slim as slim
from scipy.io import loadmat

In [2]:
import os

# Getting the current work directory (cwd)
thisdir = os.getcwd()
TrainingData= []
# r=root, d=directories, f = files
for r, d, f in os.walk('TrainingData'):
    for file in f:
        if ".mat" in file:
            print(os.path.join(r, file))
            TrainingData.append(os.path.join(r, file))

TrainingData\TrainSetPatch_1.mat
TrainingData\TrainSetPatch_10.mat
TrainingData\TrainSetPatch_11.mat
TrainingData\TrainSetPatch_12.mat
TrainingData\TrainSetPatch_13.mat
TrainingData\TrainSetPatch_14.mat
TrainingData\TrainSetPatch_15.mat
TrainingData\TrainSetPatch_16.mat
TrainingData\TrainSetPatch_17.mat
TrainingData\TrainSetPatch_18.mat
TrainingData\TrainSetPatch_19.mat
TrainingData\TrainSetPatch_2.mat
TrainingData\TrainSetPatch_20.mat
TrainingData\TrainSetPatch_3.mat
TrainingData\TrainSetPatch_4.mat
TrainingData\TrainSetPatch_5.mat
TrainingData\TrainSetPatch_6.mat
TrainingData\TrainSetPatch_7.mat
TrainingData\TrainSetPatch_8.mat
TrainingData\TrainSetPatch_9.mat


In [3]:
for i in range(0, len(TrainingData)):
    td = loadmat(TrainingData[i])['X']
    if (i==0):
        tdata = td
    else :
        tdata = np.concatenate((tdata, td), axis=1)
    
print(tdata.shape)
tdata = tdata.transpose()
print(len(tdata.transpose()[0]))

(64, 80000)
80000


In [4]:
# Deciding how many nodes wach layer should have

n_nodes_inpl = 64  #encoder
n_nodes_hl1  = 32  #encoder

n_nodes_hl2  = 32  #decoder
n_nodes_outl = 64  #decoder

In [5]:
# first hidden layer has 64*32 weights and 32 biases

hidden_1_layer_vals = {
'weights':tf.Variable(tf.random_normal([n_nodes_inpl,n_nodes_hl1])),
'biases':tf.Variable(tf.random_normal([n_nodes_hl1]))  }
print(hidden_1_layer_vals)

# second hidden layer has 32*32 weights and 32 biases

hidden_2_layer_vals = {
'weights':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
'biases':tf.Variable(tf.random_normal([n_nodes_hl2]))  }

# second hidden layer has 32*784 weights and 784 biases

output_layer_vals = {
'weights':tf.Variable(tf.random_normal([n_nodes_hl2,n_nodes_outl])),
'biases':tf.Variable(tf.random_normal([n_nodes_outl])) }

{'weights': <tf.Variable 'Variable:0' shape=(64, 32) dtype=float32_ref>, 'biases': <tf.Variable 'Variable_1:0' shape=(32,) dtype=float32_ref>}


In [6]:
# image with shape 784 goes in
input_layer = tf.placeholder('float', [None, 64])

# multiply output of input_layer wth a weight matrix and add biases

layer_1 = tf.contrib.layers.fully_connected(tf.matmul(input_layer,hidden_1_layer_vals['weights']),
                                            32,
                                            activation_fn=tf.nn.relu)

# multiply output of layer_1 wth a weight matrix and add biases

layer_2 = tf.contrib.layers.fully_connected(
       tf.add(tf.matmul(layer_1,hidden_2_layer_vals['weights']),
       hidden_2_layer_vals['biases']),32, activation_fn=tf.nn.relu)

# multiply output of layer_2 wth a weight matrix and add biases

output_layer = tf.matmul(layer_1,output_layer_vals['weights']) 

# output_true shall have the original image for error calculations

output_true = tf.placeholder('float', [None, 64])

# define our cost function
meansq =    tf.reduce_mean(tf.square(output_layer - output_true))

# define our optimizer
learn_rate = 0.1   # how fast the model should learn
optimizer = tf.train.AdagradOptimizer(learn_rate).minimize(meansq)

In [7]:
# initialising stuff and starting the session

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

# defining batch size, number of epochs and learning rate

batch_size = 500  # how many images to use together for training
hm_epochs =1000    # how many times to go through the entire dataset
tot_images =  len(tdata.transpose()[0])# total number of images

# running the model for a 1000 epochs taking 100 images in batches
# total improvement is printed out after each epoch

for epoch in range(hm_epochs):

    epoch_loss = 0    # initializing error as 0

    for i in range(int(tot_images/batch_size)):

        epoch_x = tdata[ i*batch_size : (i+1)*batch_size ]

        _, c = sess.run([optimizer, meansq],\
               feed_dict={input_layer: epoch_x, \
               output_true: epoch_x})

        epoch_loss += c

    print('Epoch', epoch, '/', hm_epochs, 'loss:',epoch_loss)
    

Epoch 0 / 1000 loss: 33.0531020332
Epoch 1 / 1000 loss: 2.57701327559
Epoch 2 / 1000 loss: 2.50986067392
Epoch 3 / 1000 loss: 2.49045878835
Epoch 4 / 1000 loss: 2.48183400277
Epoch 5 / 1000 loss: 2.47716243006
Epoch 6 / 1000 loss: 2.4743047189
Epoch 7 / 1000 loss: 2.47243312839
Epoch 8 / 1000 loss: 2.47113559954
Epoch 9 / 1000 loss: 2.47019128967
Epoch 10 / 1000 loss: 2.469486624
Epoch 11 / 1000 loss: 2.46894389577
Epoch 12 / 1000 loss: 2.46851201542
Epoch 13 / 1000 loss: 2.46816143394
Epoch 14 / 1000 loss: 2.46787037794
Epoch 15 / 1000 loss: 2.46762892883
Epoch 16 / 1000 loss: 2.46742583625
Epoch 17 / 1000 loss: 2.46725263074
Epoch 18 / 1000 loss: 2.46710237488
Epoch 19 / 1000 loss: 2.46697201673
Epoch 20 / 1000 loss: 2.46685803868
Epoch 21 / 1000 loss: 2.46675757412
Epoch 22 / 1000 loss: 2.46666874085
Epoch 23 / 1000 loss: 2.46658795793
Epoch 24 / 1000 loss: 2.46651588567
Epoch 25 / 1000 loss: 2.46645113546
Epoch 26 / 1000 loss: 2.46639174968
Epoch 27 / 1000 loss: 2.46633727755
Epoch

Epoch 226 / 1000 loss: 2.4649666287
Epoch 227 / 1000 loss: 2.46496490762
Epoch 228 / 1000 loss: 2.4649632154
Epoch 229 / 1000 loss: 2.46496138908
Epoch 230 / 1000 loss: 2.46495909151
Epoch 231 / 1000 loss: 2.46495669521
Epoch 232 / 1000 loss: 2.46495477855
Epoch 233 / 1000 loss: 2.46495307889
Epoch 234 / 1000 loss: 2.46495132893
Epoch 235 / 1000 loss: 2.46494949795
Epoch 236 / 1000 loss: 2.46494793147
Epoch 237 / 1000 loss: 2.46494297776
Epoch 238 / 1000 loss: 2.46493818983
Epoch 239 / 1000 loss: 2.46493380424
Epoch 240 / 1000 loss: 2.46493003238
Epoch 241 / 1000 loss: 2.46492651105
Epoch 242 / 1000 loss: 2.46492325049
Epoch 243 / 1000 loss: 2.46492030099
Epoch 244 / 1000 loss: 2.4649172537
Epoch 245 / 1000 loss: 2.46491424181
Epoch 246 / 1000 loss: 2.46491159406
Epoch 247 / 1000 loss: 2.46490847226
Epoch 248 / 1000 loss: 2.46490527596
Epoch 249 / 1000 loss: 2.46490192413
Epoch 250 / 1000 loss: 2.46489828266
Epoch 251 / 1000 loss: 2.46489493269
Epoch 252 / 1000 loss: 2.46489179414
Epoc

Epoch 449 / 1000 loss: 2.46413157601
Epoch 450 / 1000 loss: 2.46412640158
Epoch 451 / 1000 loss: 2.46412011608
Epoch 452 / 1000 loss: 2.46411274932
Epoch 453 / 1000 loss: 2.46410491969
Epoch 454 / 1000 loss: 2.46409721673
Epoch 455 / 1000 loss: 2.46408877615
Epoch 456 / 1000 loss: 2.46408044174
Epoch 457 / 1000 loss: 2.46407152805
Epoch 458 / 1000 loss: 2.46406290494
Epoch 459 / 1000 loss: 2.46405452397
Epoch 460 / 1000 loss: 2.46404569782
Epoch 461 / 1000 loss: 2.46403800137
Epoch 462 / 1000 loss: 2.46403164137
Epoch 463 / 1000 loss: 2.4640255142
Epoch 464 / 1000 loss: 2.46402012277
Epoch 465 / 1000 loss: 2.46401486825
Epoch 466 / 1000 loss: 2.4640114354
Epoch 467 / 1000 loss: 2.46400449425
Epoch 468 / 1000 loss: 2.46399805788
Epoch 469 / 1000 loss: 2.46399211977
Epoch 470 / 1000 loss: 2.46398640797
Epoch 471 / 1000 loss: 2.46398090571
Epoch 472 / 1000 loss: 2.46397451311
Epoch 473 / 1000 loss: 2.46396859456
Epoch 474 / 1000 loss: 2.46396162454
Epoch 475 / 1000 loss: 2.4639546508
Epoc

Epoch 671 / 1000 loss: 2.46147823799
Epoch 672 / 1000 loss: 2.46145304944
Epoch 673 / 1000 loss: 2.46142713074
Epoch 674 / 1000 loss: 2.46140113659
Epoch 675 / 1000 loss: 2.46137732733
Epoch 676 / 1000 loss: 2.46135128755
Epoch 677 / 1000 loss: 2.46132291295
Epoch 678 / 1000 loss: 2.46129598655
Epoch 679 / 1000 loss: 2.46126933116
Epoch 680 / 1000 loss: 2.46124115121
Epoch 681 / 1000 loss: 2.46121247858
Epoch 682 / 1000 loss: 2.46118256915
Epoch 683 / 1000 loss: 2.4611491356
Epoch 684 / 1000 loss: 2.46111218259
Epoch 685 / 1000 loss: 2.46107783169
Epoch 686 / 1000 loss: 2.4610411264
Epoch 687 / 1000 loss: 2.46100168675
Epoch 688 / 1000 loss: 2.46096902434
Epoch 689 / 1000 loss: 2.46092945058
Epoch 690 / 1000 loss: 2.46088873316
Epoch 691 / 1000 loss: 2.46084885485
Epoch 692 / 1000 loss: 2.46080751065
Epoch 693 / 1000 loss: 2.46076192707
Epoch 694 / 1000 loss: 2.46071268898
Epoch 695 / 1000 loss: 2.460659693
Epoch 696 / 1000 loss: 2.46060473658
Epoch 697 / 1000 loss: 2.46054627839
Epoch

Epoch 894 / 1000 loss: 2.42303235922
Epoch 895 / 1000 loss: 2.42292248085
Epoch 896 / 1000 loss: 2.42281843629
Epoch 897 / 1000 loss: 2.42271766067
Epoch 898 / 1000 loss: 2.42262098659
Epoch 899 / 1000 loss: 2.42252686899
Epoch 900 / 1000 loss: 2.42243206222
Epoch 901 / 1000 loss: 2.42234020494
Epoch 902 / 1000 loss: 2.42224829923
Epoch 903 / 1000 loss: 2.42216014583
Epoch 904 / 1000 loss: 2.42206925433
Epoch 905 / 1000 loss: 2.42198222782
Epoch 906 / 1000 loss: 2.42189576477
Epoch 907 / 1000 loss: 2.42180787493
Epoch 908 / 1000 loss: 2.42172090337
Epoch 909 / 1000 loss: 2.42163479608
Epoch 910 / 1000 loss: 2.42154883221
Epoch 911 / 1000 loss: 2.42146343458
Epoch 912 / 1000 loss: 2.42137541529
Epoch 913 / 1000 loss: 2.42128983792
Epoch 914 / 1000 loss: 2.42120154761
Epoch 915 / 1000 loss: 2.42111262586
Epoch 916 / 1000 loss: 2.42102341633
Epoch 917 / 1000 loss: 2.42093397398
Epoch 918 / 1000 loss: 2.4208434606
Epoch 919 / 1000 loss: 2.42075266782
Epoch 920 / 1000 loss: 2.42066338658
Ep

In [8]:
# pick any image

any_image = all_images[999]

# run it though the autoencoder

output_any_image = sess.run(output_layer,\
                   feed_dict={input_layer:[any_image]})

# run it though just the encoder

encoded_any_image = sess.run(layer_1,\
                   feed_dict={input_layer:[any_image]})

# print the original image

plt.imshow(any_image(28,28),  cmap='Greys')
plt.show()

# print the encoding

print(encoded_any_image)

NameError: name 'all_images' is not defined